# Seminario 3. Programación funcional en Python: comprehensions, iteradores y generadores.

En este seminario se presentan varios mecanismos que Python proporciona para la creación de secuencias de datos: las *comprehensions*, los *iterables*, los *iteradores* y los *generadores*. Al igual que las funciones `map`, `filter` y `reduce` presentadas en un seminario anterior, todos estos mecanismos provienen de los *lenguajes funcionales*. Python, aunque por diseño es un lenguaje orientado a objetos, incorpora conceptos y herramientas que provienen de lenguajes imperativos (estructurados) y de los lenguajes funcionales, lo que realmente lo convierte en un lenguaje multiparadigma.

## Comprehensions

Las llamadas *comprehensions* son una forma eficiente y concisa de crear e inicializar determinados contenedores: listas, diccionarios y conjuntos. Se caracterizan por ser compactas y replicar aproximadamente la notación matemática que suele utilizarse para definir dichas estructuras.

## List comprehensions

La sintaxis general de una list comprehension es:

    [ expresión  for item_1 in iterable_1 if condición_1
                 for item_2 in iterable_2 if condición_2
                                ...
                 for item_N in iterable_N if condición_N ]

Como puede verse, la sintaxis es muy similar a la notación matemática utilizada habitualmente para definir este tipo de estructuras:

$\qquad l = \{ x^2\ |\ x \in \mathbb{N}, 1 \leq x \leq 10\}$

     l = [x*x for x in range(1,11)]

Además de elegante, esta forma de crear listas es más rápida que el procedimiento habitual mediante un ciclo for. Veamos algunos ejemplos:

In [1]:
import random
from pprint import pprint

nums = [random.randint(-10,10) for i in range(10)]
facts = [random.gauss(0, 4) for i in range(10)]
pprint(nums)
pprint(facts)

[2, -8, 6, -10, -6, -8, 8, -5, -8, -2]
[0.4367924129123529,
 0.09074764637165093,
 -3.3305089202770013,
 -2.230182830465015,
 0.2526912431931702,
 1.2381111506748113,
 -2.4128922312294687,
 6.274194523364716,
 -2.3419072682451345,
 8.526030530231788]


In [2]:
sqr = [x*x for x in nums if x>0]
sqr

[4, 36, 64]

In [3]:
prod = [x*y for x in nums if x%3==0 and x!=0 for y in facts if y>0]
prod

[2.6207544774741174,
 0.5444858782299056,
 1.5161474591590212,
 7.4286669040488675,
 37.64516714018829,
 51.15618318139073,
 -2.6207544774741174,
 -0.5444858782299056,
 -1.5161474591590212,
 -7.4286669040488675,
 -37.64516714018829,
 -51.15618318139073]

Podemos anidar una list comprehension dentro de otra:

In [4]:
m = [[random.randint(0,1) for j in range(4)] for i in range(3)]
m

[[1, 0, 1, 0], [0, 0, 0, 0], [1, 0, 0, 0]]

In [5]:
t = [[m[i][j] for i in range(len(m))] for j in range(len(m[0]))]
t

[[1, 0, 1], [0, 0, 0], [1, 0, 0], [0, 0, 0]]

También podemos aplicar este método para escribir o leer datos de un archivo:

In [6]:
# primero escribimos la matriz m en el fichero "matriz.txt"
with open("matriz.txt", "w", encoding="utf-8") as fp:
    for fila in m:
        fp.write(' '.join([str(elem) for elem in fila])+'\n')

# a continuación, leemos el fichero "matriz.txt" para crear una copia de la matriz m
with open("matriz.txt", "r", encoding="utf-8") as fp:
    m_copy = [[int(elem) for elem in linea.split()] for linea in fp]
m_copy

[[1, 0, 1, 0], [0, 0, 0, 0], [1, 0, 0, 0]]

## Dictionary comprehensions



La sintaxis general de una *dictionary comprehension* puede tener dos formas:

    { k: func(k) for k in secuencia if condición }
    
    { k_expr: v_expr for k, v in secuencia_tuplas if condición }

Dos ejemplos:

In [7]:
d = {i: None for i in range(1, 100) if i%17==0}
d

{17: None, 34: None, 51: None, 68: None, 85: None}

In [8]:
palabras = ["calcio", "magnesio", "potasio", "sodio", "hierro", "fósforo"]
d = {w: len(w) for w in palabras}
d

{'calcio': 6,
 'magnesio': 8,
 'potasio': 7,
 'sodio': 5,
 'hierro': 6,
 'fósforo': 7}

También podemos anidar una *dictionary comprehension* dentro de otra:

In [9]:
import random
import string

d = {i: {j: random.choice(string.ascii_letters) for j in range(i)} for i in range(5)}
d

{0: {},
 1: {0: 't'},
 2: {0: 'Z', 1: 'r'},
 3: {0: 'O', 1: 'O', 2: 'O'},
 4: {0: 'e', 1: 'W', 2: 'g', 3: 'V'}}

## Set comprehensions

Por supuesto, para crear e inicializar conjuntos podemos usar *set comprehensions*. Veamos algunos ejemplos.

In [10]:
import string

cadena = "(¡¡¡Los pasteles vinieron de [[Francia]], mon ami!!!)"
blanks = set(string.punctuation+string.whitespace)
s = {c.lower() for c in cadena if c not in blanks}
print(s)

{'v', 'c', 'a', 't', 'l', 'p', 's', '¡', 'n', 'r', 'f', 'i', 'm', 'e', 'o', 'd'}


In [11]:
import random

s = {n for n in random.choices(range(1,101,2), k=10)} 
print(s)

{5, 69, 71, 73, 11, 13, 79, 47, 93}


In [12]:
lst = [('Manu', 19, 'Gernika'), ('Jone', 21, 'Vitoria'), ('Martina', 19, 'Bilbao'), ('Mikel', 22, 'Eibar')]
s = {c for n, e, c in lst}
s

{'Bilbao', 'Eibar', 'Gernika', 'Vitoria'}

## Iterables y contextos de iteración.

Al inicio del aprendizaje de Python se introduce el concepto de *secuencia*, como son las cadenas, listas, tuplas, o archivos; se dice también que la función `range` crea una secuencia numérica; y se explica que la forma natural de recorrer una secuencia de inicio a fin es utilizar la sentencia `for` del siguiente modo:

    for variable in secuencia:
        ...

Sin embargo, se trata de una simplificación de un concepto mucho más general: el de **iterable**. Un objeto iterable es aquél que permite realizar una iteración sobre todos sus elementos. Las secuencias son obviamente iterables, pero hay muchos otros objetos que, no siendo secuencias, también son iterables.

Todo objeto iterable se puede utilizar en un **contexto de iteración**, como es el caso de la sentencia `for`. De modo que, en realidad, un `for` tiene esta forma:

    for variable in iterable:
        ...

En Python existen numerosos ejemplos de contextos de iteración. Por ejemplo, los desempaquetamientos, sean implícitos (como en una asignación múltiple) o explícitos (al utilizar los operadores `*` y `**`) definen contextos de iteración. También los encontramos (aunque de forma más evidente) en las *comprehensions* y en las funciones y expresiones generadoras que trataremos a continuación.

## Iterables e iteradores.

Formalmente, un objeto es *iterable* si posee un método `__iter__`. Dicho método ha de retornar un objeto **iterador**, que se caracteriza por utilizar el denominado *protocolo de iterador* (*iterator protocol*), el cual consiste en:

- Poseer un método `__next__`, que produce y retorna un elemento a partir del iterable. Si no quedan más elementos, el iterador desencadena la excepción `StopIteration`.
- Poseer un método `__iter__` que retorna el mismo objeto iterador. Esto implica que todo iterador es también un iterable.

En definitiva: *el objeto iterador itera sobre los elementos del iterable del que procede*.

El método `__next__` del iterador producirá los elementos del iterable en un orden concreto: si el iterable es una secuencia, utilizará el orden de dicha secuencia; pero si el iterable no define un orden para sus elementos, no debemos realizar suposiciones sobre el orden de producción por parte de su iterador.

Por otra parte, un iterador producirá elementos a partir del iterable del que procede, pero *no tienen por qué ser exactamente los elementos del iterable*. Por ejemplo, los iteradores sobre diccionarios producen sólo las claves.

Python ofrece dos funciones específicas sobre iteradores: `iter` y `next`, que invocan respectivamente los métodos `__iter__` y `__next__` del objeto que se les pasa como argumento, y que retornan lo mismo que sendos métodos.

## Ejemplos de uso de iterables e iteradores

Empecemos por definir una lista `l`. Como ya hemos dicho, las listas son objetos iterables; es por esa razón que las podemos recorrer dentro de contextos de iteración, como un `for`:

In [13]:
l = ['hola', 'qué', 'tal']
for c in l:
    print(c)

hola
qué
tal


Ahora veamos qué sucede en el `for` con más detalle. Lo primero que sucede en un contexto de iteración es que Python invoca el método `__iter__` del objeto iterable (en este ejemplo, la lista `l`). Nosotros lo haremos utilizando la función `iter` por simple comodidad:

In [14]:
it = iter(l)
print(it)

Obsérvese que `iter(l)` ha retornado un objeto iterador (de clase `list_iterator`) que itera sobre la lista `l`. Como todo iterador, este objeto cuenta con un método `__next__` que, al invocarlo, retornará un nuevo elemento de `l`. Dado que se trata de una lista, sus elementos tienen un orden definido, por lo que el iterador `it` los producirá en ese mismo orden.

A continuación invocaremos dicho método, utilizando la función `next`:

In [15]:
print(next(it))
print(next(it))
print(next(it))

hola
qué
tal


Nótese que el iterador `it` retiene el estado tras cada invocación de su método `__next__`: es decir, *recuerda* en qué punto de la secuencia ha quedado. De modo que el iterador puede llegar a *consumirse*, es decir, terminar de recorrer la secuencia. ¿Qué sucede entonces? Ya hemos obtenido los tres elementos de la lista `l`. Veamos qué sucede al ejecutar `next(it)` una vez más:

In [16]:
print(next(it))

StopIteration: 

Como podemos comprobar, se produce una excepción de tipo `StopIteration`. Aquí vemos el mensaje de error porque no se ha capturado esa excepción, pero en los contextos de iteración sí se captura, precisamente para salir de dicho contexto cuando al iterador ya no le quedan elementos por producir. En una sentencia `for`, la salida del contexto de iteración consiste simplemente en finalizar el ciclo. De hecho, podemos simular un `for` como el de más arriba fácilmente:

> Nota: aunque el manejo de excepciones en Python se tratará en un seminario posterior, el uso que realizamos a continuación debería entenderse fácilmente.

In [17]:
it = iter(l)
while True:
    try:
        c = next(it)
    except StopIteration:
        break
    print(c)

hola
qué
tal


El funcionamiento de un iterador puede verse afectado por posibles modificaciones al iterable del que procede. Esto sucede porque el objeto iterador utiliza los elementos del objeto iterable. Ahora bien, una vez que se ha agotado el iterador (habiéndose desencadenado `StopIteration`), su estado es irreversible. En cualquier caso, no es prudente modificar un iterable mientras se utiliza un iterador creado a partir de él.

## Iteradores sobre diccionarios.

Como ya sabemos, los objetos de clase `dict` cuentan con los métodos `keys`, `values` e `items`, los cuales retornan lo que se conoce como *vistas* (*views*) sobre las claves, los valores y tuplas clave-valor, respectivamente. Dichas *vistas* no consumen memoria adicional. Por tanto, podemos iterar sobre un diccionario de forma muy sencilla:

In [18]:
d = {'a': 1, 'b': 2, 'c': 3}

for k in d.keys():
    print(k)
    
for v in d.values():
    print(v)
    
for i in d.items():
    print(i)

a
b
c
1
2
3
('a', 1)
('b', 2)
('c', 3)


Sucede, no obstante, que los objetos `dict` son en sí mismos iterables:

In [19]:
it = iter(d)
print(next(it))
print(next(it))
print(next(it))

a
b
c


Como podemos ver, el método `__next__` de un iterador no está obligado a producir elementos del iterable: puede diseñarse para que retorne lo que se considere más apropiado. Así, los iteradores sobre diccionarios producen sus claves (desde Python 3.7, el orden de las claves de un diccionario es el de su inserción). De modo que si simplemente deseamos iterar sobre las claves de un diccionario, podemos prescindir del método `keys`:

In [20]:
for k in d:
    print(k)

a
b
c


## Otros objetos iterables.

No sólo los objetos de las clases contenedoras son iterables; existen muchos tipos de objetos que también poseen el método `__iter__`. Por ejemplo, la función `range` crea y retorna un objeto iterable de clase `range` que, en realidad, no contiene una estructura de datos como una lista o tupla con todos los elementos del rango, sino *la forma de producir esos elementos*. Lógicamente, un iterador creado a partir de un objeto `range` produce secuencialmente cada elemento del rango cuando se invoca su método `__next__`.

In [21]:
r = range(10)
it = iter(r)
print('r:', r)
print('iter(r):', it)
print(next(it), next(it), next(it))

r: range(0, 10)
iter(r): <range_iterator object at 0x7f712c1facc0>
0 1 2


También existen funciones que crean *directamente* objetos iteradores que, por tanto, no proceden de un iterable. Por ejemplo, la función `zip` sirve para emparejar (o “n-tuplar”, en realidad) los elementos de dos o más secuencias… o iterables. Pues bien, esta función *retorna directamente un iterador*; obsérvese a continuación que no es necesario utilizar el método `__iter__` para crear un iterador a partir del objeto que retorna `zip`:

In [22]:
z = zip([1,2,3],[4,5,6])
print(next(z), next(z), next(z))

(1, 4) (2, 5) (3, 6)


Recuérdese que se puede utilizar un iterador allí donde se pueda emplear un iterable, por lo que podemos utilizar `zip` en un `for`:

In [23]:
for t in zip(range(0,5),range(1,6),range(2,7)):
    print(t)

(0, 1, 2)
(1, 2, 3)
(2, 3, 4)
(3, 4, 5)
(4, 5, 6)


In [24]:
m = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]
t = [list(t) for t in zip(*m)] # traspuesta de m
t

[[1, 5, 9], [2, 6, 10], [3, 7, 11], [4, 8, 12]]

Obsérvese que, de forma similar a `range`, el iterador construido por la función `zip` no contiene todos los elementos, sino la forma de producirlos. En este último ejemplo en el que se le pasan como argumentos tres objetos `range`, el uso de memoria es mínimo, ya que ni los objetos `range` (ni los iteradores que se construyen sobre ellos), ni el objeto `zip` contienen elemento alguno (enteros o tuplas), sino solamente el código necesario para producirlos en el momento en que se invoque el método `__next__`. En consecuencia, podemos afirmar que **el uso de iteradores resulta extremadamente eficiente en cuanto a consumo de memoria**.

## Funciones generadoras.

Las funciones generadoras son una herramienta que posibilita crear iteradores de forma sencilla y flexible. En su forma más general, se definen de forma casi idéntica a una función convencional, salvo que utilizan la sentencia `yield` para *retornar* datos. La diferencia entre las sentencias `yield` y `return` es que la primera hace que el generador conserve su estado, de forma que la próxima vez que se invoca `__next__` continúa en el punto en que se quedó tras ejecutarse `yield`, recordando también todas sus variables locales.

Veamos un ejemplo de función generadora que produce una secuencia correspondiente a la serie de Fibonacci. La función recibe como argumento el número de elementos deseados.

In [25]:
def Fibogen(nterms=None):
    n = 0
    a, b = 0, 1
    
    while nterms == None or n < nterms:
        n += 1
        a, b = b, a+b
        yield a

Los métodos `__next__` e `__iter__` se crean automáticamente y la excepción `StopIteration` se desencadena también automáticamente al salir de la función. El uso de esta función es muy sencillo:

In [26]:
for n in Fibogen(10):
    print(n)

1
1
2
3
5
8
13
21
34
55


In [27]:
l = list(Fibogen(20))
print('l =', l)

l = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]


El cociente entre un término de la serie de Fibonacci y el anterior tiene como límite la *razón aúrea* (𝜑). Supongamos que queremos (por puro masoquismo) calcular 𝜑 de ese modo, en lugar utilizar directamente la fórmula 𝜑=(1+√5)/2. Con 50 términos ya se consigue muy buena precisión. La cuestión es cómo conseguir *sólo* los dos últimos términos generados por `Fibogen(50)` sin almacenarlos previamente en una lista u otra estructura, ya que eso consumiría memoria sin necesidad.

In [28]:
# Cálculo directo
print(f'Usando la fórmula: 𝜑 = {(1+5**0.5)/2}')

# Consumiendo memoria
a, b = list(Fibogen(50))[-2:]
print(f'Usando una lista: 𝜑 = {b/a}')

# Consumiendo memoria, pero de forma más elegante
*foo, a, b = Fibogen(50)
del foo
print(f'Usando un empaquetamiento: 𝜑 = {b/a}')

# Usando la memoria justa y necesaria
fibo_it = Fibogen(50)
b = next(fibo_it)
for c in fibo_it:
    a = b
    b = c
print(f'Usando un for con ‘next’ previo: 𝜑 = {b/a}')

Usando la fórmula: 𝜑 = 1.618033988749895
Usando una lista: 𝜑 = 1.618033988749895
Usando un empaquetamiento: 𝜑 = 1.618033988749895
Usando un for con ‘next’ previo: 𝜑 = 1.618033988749895


Las funciones generadoras pueden ser extremadamente útiles para escribir programas basados en filtros que operan sobre flujos de datos. También son útiles en operaciones que requieren el manejo de iterables complejos. Por ejemplo:

In [29]:
# Devuelve alternativamente 1 y -1
def plusminus():
    pm = 1
    while True:
        yield pm
        pm = -pm

# En cada llamada devuelve el siguiente minor de la matriz a (desarrollando sobre a[0])
def minors(a):
    for i in range(len(a)):
        yield [[fila[j] for j in range(len(fila)) if j!=i] for fila in a[1:]]

# Devuelve el determinante de una matriz cuadrada a
def det(a):
    if len(a) == 1:
        return a[0][0]
    else:
        return sum(x*y*det(z) for x,y,z in zip(plusminus(), a[0], minors(a)))

mat = [[1, 3, -5],[1, 2, 0], [-2, 2, 4]]
print(det(mat))

-34


## Expresiones generadoras.

Cuando las funciones generadoras son muy sencillas, se pueden definir de manera compacta en forma de *expresiones generadoras*. Su sintaxis es prácticamente idéntica a la de las *list comprehensions*, pero utilizando paréntesis en lugar de corchetes:

    ( expresión  for item_1 in iterable_1 if condición_1
                 for item_2 in iterable_2 if condición_2
                                ...
                 for item_N in iterable_N if condición_N )

Los paréntesis no son necesarios si la expresión generadora es el único argumento de una función. A diferencia de una list comprehension, una expresión generadora no almacena datos en memoria, sino que los genera a demanda. Por tanto, las expresiones generadoras se pueden utilizar en lugar de *list comprehensions* cuando no necesitamos crear una lista como tal, sino tan sólo iterar sobre sus elementos. Por ejemplo:

In [30]:
s1 = sum([n*n for n in range(10)]) # con list comprehension, consumiendo memoria
s2 = sum(n*n for n in range(10)) # con expresión generadora, sin consumir memoria
print(s1)
print(s2)

285
285


La función `sum` espera un *iterable* como argumento. La diferencia es que con la expresión generadora no se construye lista alguna, por lo que resulta más eficiente en tiempo y en uso de memoria.

En el siguiente ejemplo se calcula un producto escalar de dos vectores (listas) sin necesidad de construir ninguna lista adicional:

In [31]:
vec1 = [10, 20, 30, 40, 50]
vec2 = [2, 3, 5, 7, 11]
print(sum(x*y for x, y in zip(vec1, vec2)))

1060


La función `sum` utiliza la expresión generadora que se le ha pasado como argumento para ir obteniendo los sumandos uno por uno, que va acumulando en el sumatorio. A su vez, cada vez que se le demanda un nuevo elemento desde la función, la función generadora obtiene del iterador creado por `zip` una nueva tupla, de la cual extrae y multiplica sus dos componentes; y a su vez, el iterador del `zip` obtiene un par de elementos de ambos vectores y forma con ellos una tupla a medida que se le va demandando por parte de la expresión generadora. Éste es un proceso muy eficiente en términos de uso de memoria y tiempo.

Quizá se aprecie mejor la potencia y facilidad de uso de los iteradores, funciones y expresiones generadoras si descomponemos la expresión del producto escalar en tres funciones ( `mi_sum`, `prod_elems`y `mi_zip`) que emulan el comportamiento de la expresión mostrada arriba:

In [32]:
def mi_sum(iterable):
    suma = 0
    for item in iterable:
        suma += item
    return suma
    
def prod_elems(v1, v2):
    for x, y in mi_zip(v1, v2):
        yield x*y
    
def mi_zip(*iterables):
    iteradores = [iter(it) for it in iterables]
    while True:
        try:
            t = tuple([next(i) for i in iteradores])
            yield t
        except StopIteration:
            break

print(mi_sum(prod_elems(vec1, vec2)))

1060


A continuación se muestra otro ejemplo en el que primero creamos un archivo con datos de un inventario, para después leerlo por medio de dos expresiones generadoras acopladas. Obsérvese cómo la función `sum` de Python pone en marcha la expresión generadora `costes`, que a su vez pone en marcha la expresión genradora `items`, que es la que recorre el archivo:

In [33]:
with open("inventario.txt", "w", encoding="utf-8") as fp:
    fp.write("mesas 20 33.24\nsillas 15 12.67\nbombillas 9 2.54\nlapiceros 213 0.87")
    
with open("inventario.txt", "r", encoding="utf-8") as fp:
    items = (linea.split() for linea in fp)
    costes = (int(f[1])*float(f[2]) for f in items)
    total = sum(costes)

print(f'Total: {total:.2f}')

Total: 1063.02


## El módulo `itertools`.

La biblioteca estándar de Python cuenta con un módulo llamado `itertools`, que ofrece un conjunto de iteradores que, en conjunto, forman un “álgebra de iteradores“ con los que se pueden realizar construcciones compactas y extremadamente potentes; más aún si se combinan con el módulo `operator` de la biblioteca estándar.

No vamos a tratar aquí el contenido y uso del módulo `itertools`, pero recomendamos [la lectura de su documentación](https://docs.python.org/3/library/itertools.html), que además incluye diversos ejemplos de uso.